In [1]:
import os
os.getcwd()
os.chdir("../")

In [2]:
os.getcwd()

'd:\\workspace\\git_projects\\NBFI-Loan-Repayment'

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
from datetime import datetime
from utils.dbmanager import create_sql_connection, ingest_data_to_sql, query_data
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
from scipy import stats
import random
random.seed(42)

['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [4]:
server = 'nbfi-loan-def-prediction.database.windows.net'     
database = 'NBFIDB'  
username = 'adminuser'       
password = 'Rambo@12345' 

# Establish connection
conn = create_sql_connection(server, database, username, password)

Connection established successfully!


In [5]:
data = query_data(conn, "SELECT * FROM bronze.CustomerRaw")
data.head(5)

Query executed in: 0.4776 seconds


,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters
0,12100111,18900.0,0,0,0,1,0,108042.3,4297.95,Alone,Retired,Secondary,M,Male,CL,Home,0.000000,23490.0,365243.0,13884.0,4644.0,0.0,1,0,0,Unknown,2,2.0,1,12,Yes,Yes,XNA,0.000000,0.689242,0.475850,0.0,1294.0,8.0,0
1,12100205,6750.0,1,0,1,1,0,18000.0,900.00,Alone,Service,Secondary,M,Male,RL,Home,0.007330,14420.0,7675.0,6653.0,5158.0,13.0,1,0,0,Medicine,3,2.0,5,11,Yes,Yes,Medicine,0.000000,0.517620,0.703203,0.0,0.0,1.0,0
2,12100223,13500.0,1,1,0,1,0,18000.0,900.00,Alone,Service,Secondary,S,Male,RL,Home,0.002042,11891.0,1126.0,5641.0,0.0,1.0,1,0,0,Cooking,1,3.0,3,9,No,Yes,Restaurant,0.000000,0.360573,0.000000,0.0,448.0,0.0,0
3,12100229,11250.0,0,1,1,1,0,112950.0,3745.80,Relative,Commercial,Secondary,M,Female,CL,Home,0.028663,20322.0,1967.0,3178.0,3184.0,0.0,1,0,0,Managers,2,2.0,1,15,Yes,Yes,Agriculture,0.000000,0.537326,0.694093,0.0,1069.0,6.0,0
4,12100330,13500.0,0,0,0,1,0,9000.0,1067.85,Alone,Service,Graduation,S,Male,CL,Family,0.035792,8182.0,892.0,8182.0,846.0,9.0,1,1,0,Laborers,1,2.0,3,18,Yes,Yes,Industry: type 9,0.298041,0.587561,0.367291,0.0,256.0,0.0,0


In [7]:
data = pd.read_csv(r'data/NBFIData.csv')
data.shape

(121787, 40)

In [8]:
data.head()

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,Client_Housing_Type,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Own_House_Age,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Occupation,Client_Family_Members,Client_City_Rating,Application_Process_Day,Application_Process_Hour,Client_Permanent_Match_Tag,Client_Contact_Work_Tag,Type_Organization,Score_Source_1,Score_Source_2,Score_Source_3,Social_Circle_Default,Phone_Change,Credit_Bureau,Defaulters
0,12142509,6750.0,0.0,0.0,1.0,0.0,0.0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,Home,0.028663,13957.0,1062.0,6123.0,383.0,NaN,1,1,0,Sales,2.0,2.0,6.0,17.0,Yes,Yes,Self-employed,0.568066,0.478787,NaN,0.0186,63.0,NaN,0
1,12138936,20250.0,1.0,0.0,1.0,NaN,0.0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,Home,0.008575,14162.0,4129.0,7833.0,21.0,0.0,1,0,1,NaN,2.0,2.0,3.0,10.0,Yes,Yes,Government,0.563360,0.215068,NaN,NaN,NaN,NaN,0
2,12181264,18000.0,0.0,0.0,1.0,0.0,1.0,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,Family,0.022800,16790.0,5102.0,NaN,331.0,NaN,1,0,0,Realty agents,2.0,2.0,4.0,NaN,Yes,Yes,Self-employed,NaN,0.552795,0.329655,0.0742,277.0,0.0,0
3,12188929,15750.0,0.0,0.0,1.0,1.0,0.0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,Home,0.010556,23195.0,365243.0,NaN,775.0,NaN,1,0,0,NaN,2.0,3.0,2.0,15.0,Yes,Yes,XNA,NaN,0.135182,0.631355,NaN,1700.0,3.0,0
4,12133385,33750.0,1.0,0.0,1.0,0.0,2.0,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,Home,0.020713,11366.0,2977.0,5516.0,4043.0,6.0,1,0,0,Laborers,4.0,1.0,3.0,NaN,Yes,Yes,Business Entity Type 3,0.508199,0.301182,0.355639,0.2021,674.0,1.0,0


In [9]:
# Calculate missing value percentage
missing_percentage = (data.isnull().sum() / len(data)) * 100

# Display the result
missing_percentage = missing_percentage[missing_percentage > 0].sort_values(ascending=False)
print(missing_percentage)

Own_House_Age                 65.729511
Score_Source_1                56.484682
Social_Circle_Default         50.816590
Client_Occupation             34.002808
Score_Source_3                22.096776
Credit_Bureau                 15.217552
ID_Days                        4.897896
Score_Source_2                 4.665523
Population_Region_Relative     3.988110
Loan_Annuity                   3.950340
Client_Income_Type             3.037270
Client_Housing_Type            3.026596
Phone_Change                   3.006068
House_Own                      3.003605
Application_Process_Hour       3.001141
Employed_Days                  2.996215
Loan_Contract_Type             2.994572
Client_Education               2.991288
Child_Count                    2.984719
Active_Loan                    2.983077
Credit_Amount                  2.981435
Bike_Owned                     2.975687
Registration_Days              2.967476
Type_Organization              2.961728
Client_Income                  2.960086


In [10]:
columns_not_useful= ['Own_House_Age', 'Score_Source_1', 'Social_Circle_Default', 'Client_Occupation']
data[columns_not_useful] = data[columns_not_useful].astype(str).fillna("NA")

### Misiing Value Treatment

In [11]:
data['Loan_Annuity'] = data['Loan_Annuity'].astype(float)

In [12]:
# Numerical columns -> Filling with the median value
num_cols = data.select_dtypes(include=['int', 'float']).columns
num_cols

Index(['ID', 'Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan',
       'House_Own', 'Child_Count', 'Credit_Amount', 'Loan_Annuity',
       'Population_Region_Relative', 'Age_Days', 'Employed_Days',
       'Registration_Days', 'ID_Days', 'Mobile_Tag', 'Homephone_Tag',
       'Workphone_Working', 'Client_Family_Members', 'Client_City_Rating',
       'Application_Process_Day', 'Application_Process_Hour', 'Score_Source_2',
       'Score_Source_3', 'Phone_Change', 'Credit_Bureau', 'Defaulters'],
      dtype='object')

In [13]:
def impute_missing_values(data, numerical_cols):
    """
    Impute missing values for numeric columns using mean or median based on business context.

    Parameters:
        data (pd.DataFrame): Input DataFrame with missing values.
        numerical_cols (list): List of numeric column names.

    Returns:
        pd.DataFrame: DataFrame with imputed values.
    """
    # Variables to impute with mean
    mean_cols = [
        "Client_Income", "Credit_Amount", "Loan_Annuity",
        "Population_Region_Relative", "Score_Source_2", "Score_Source_3"
    ]

    # Variables to impute with median
    median_cols = [
        col for col in numerical_cols if col not in mean_cols
    ]

    # Impute mean columns
    for col in mean_cols:
        if col in data.columns:
            data[col].fillna(data[col].mean(), inplace=True)

    # Impute median columns
    for col in median_cols:
        if col in data.columns:
            data[col].fillna(data[col].median(), inplace=True)

    return data

In [14]:
# Apply the function
data = impute_missing_values(data, num_cols)

In [15]:
data['Application_Process_Hour'] = data['Application_Process_Hour'].replace('#NA', pd.NA).fillna('12').astype('int')

In [16]:
# categorical columns-> Filling with the mode value
cat_cols = ['Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Marital_Status', 'Client_Gender', 'Loan_Contract_Type',
       'Client_Housing_Type', 'Type_Organization']

In [17]:
def impute_categorical_columns(data, categorical_cols):
    """
    Impute missing values in categorical columns using the mode (most frequent value).

    Parameters:
        data (pd.DataFrame): The DataFrame containing categorical data.
        categorical_cols (list): List of categorical column names.

    Returns:
        pd.DataFrame: DataFrame with imputed categorical values.
    """
    for col in categorical_cols:
        if col in data.columns:
            # Impute missing values with mode (most frequent value)
            mode_value = data[col].mode()[0]  # Get the most frequent value
            data[col].fillna(mode_value, inplace=True)
            print(f"Missing values in '{col}' imputed with mode: {mode_value}")
    
    return data

In [18]:
# Applying the function
data = impute_categorical_columns(data, cat_cols)

Missing values in 'Accompany_Client' imputed with mode: Alone
Missing values in 'Client_Income_Type' imputed with mode: Service
Missing values in 'Client_Education' imputed with mode: Secondary
Missing values in 'Client_Marital_Status' imputed with mode: M
Missing values in 'Client_Gender' imputed with mode: Male
Missing values in 'Loan_Contract_Type' imputed with mode: CL
Missing values in 'Client_Housing_Type' imputed with mode: Home
Missing values in 'Type_Organization' imputed with mode: Business Entity Type 3


In [19]:
data.isna().sum()

ID                            0
Client_Income                 0
Car_Owned                     0
Bike_Owned                    0
Active_Loan                   0
House_Own                     0
Child_Count                   0
Credit_Amount                 0
Loan_Annuity                  0
Accompany_Client              0
Client_Income_Type            0
Client_Education              0
Client_Marital_Status         0
Client_Gender                 0
Loan_Contract_Type            0
Client_Housing_Type           0
Population_Region_Relative    0
Age_Days                      0
Employed_Days                 0
Registration_Days             0
ID_Days                       0
Own_House_Age                 0
Mobile_Tag                    0
Homephone_Tag                 0
Workphone_Working             0
Client_Occupation             0
Client_Family_Members         0
Client_City_Rating            0
Application_Process_Day       0
Application_Process_Hour      0
Client_Permanent_Match_Tag    0
Client_C

In [ ]:
# Ingest Data into Bronze Table
if conn:
    ingest_data_to_sql(conn, data.iloc[:1001,:], table_name="CustomerRaw", schema="Silver", batch_size=1000)
    #conn.close()  # Close connection after insertion

In [20]:
data.to_csv(r'data/interim/NBFIDataSilver.csv', index=False)